## Problem 2

In [1]:
# Imports
import numpy as np
import scipy
import math
import matplotlib.pyplot as plt
import statistics
import pandas as pd
from IPython.display import Image

In [2]:
# data
la = 5 # lambda, Demand rate [per day]
mu = 1/3 # arrival rate [per day]
r = 20 # reorder point
Q = 20 # Order quantity
cF = 100 # Fixed cost per order
cH = 2 # Holding cost per item per day
cS = 230 # Stock-out cost per day
L = 1/mu

#### a) 

![Markov Transition Diagram](Markov_Task2a.jpg)

In [3]:
s0 = 0 #Initial state
A = np.zeros((41, 41))
# Set arrival rate
for i in range(r + 1):
    A[i, i + Q] = mu
# Set demand rate 
for i in range(1, 41):
    A[i, i - 1] = la # ???
# Set diagonals such that each row equals 0
for i in range(41):
    for j in range(41):
        if i == j:
            diag_val = -np.sum(A[i])
            A[i,j] = diag_val
print(f"row 0 = \n{A[0]}")
print(f"row 1 = \n{A[1]}")

print(f"row 20 = \n{A[20]}")
print(f"row 40 = \n{A[-1]}")

row 0 = 
[-0.33333333  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.33333333  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.        ]
row 1 = 
[ 5.         -5.33333333  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.33333333  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.        ]
row 20 = 
[ 0.          0.          0.          0.          0.          0.
  0

In [4]:
# print("A = ")
# for i in range(41):
#     string = ""
#     for j in range(41):
#         if A[i,j] == 0:
#             string += " 0 "
#         else:
#             string += " " + str(A[i,j])[:3]
#     print(string)
#     print("")
    

#### b) 

To find the steady state solution, we follow the procedure in chapter **7.1.3**. We have the $A$ matrix, and substitute the first column with 1's to get $A_1$. $b$ is simply a column vector with 0s in all places but the first. In matrix form, we have $$P \cdot A_1 = b $$ We transpose both sides of the equation to get $$A_1^T \cdot P^T = b^T$$ This is on standard form, and we can solve it using scipy.linalg.solve.

In [5]:
# Replace first column with 1's in the A-matrix to get A1
A[:,0] = 1
b = np.zeros(41)
b[0] = 1
# b = np.array([b])

# print(At)
PT = scipy.linalg.solve(np.transpose(A), b.T)
P = PT.T
print(P)
# print(f"row 0 = \n{A[0]}")
# print(f"row 20 = \n{A[20]}")
# print(f"row 40 = \n{A[-1]}")

[0.17101476 0.01140098 0.01216105 0.01297179 0.01383657 0.01475901
 0.01574294 0.01679247 0.01791197 0.0191061  0.02037984 0.0217385
 0.02318773 0.02473358 0.02638249 0.02814132 0.03001741 0.03201857
 0.03415314 0.03643002 0.03885868 0.03004828 0.02928821 0.02847748
 0.02761269 0.02669025 0.02570632 0.02465679 0.02353729 0.02234316
 0.02106942 0.01971076 0.01826153 0.01671568 0.01506677 0.01330794
 0.01143185 0.00943069 0.00729612 0.00501925 0.00259058]


In [6]:
ave_time_no_prod = P[0]
print(f"Average time we are not able to produce product 2 = {ave_time_no_prod*100:.2f} % of the time")

Average time we are not able to produce product 2 = 17.10 % of the time


#### c) 

For a stock-out situation to occur, we will have 0 parts left and have a customer that wants a unit. 

In [7]:
stock_out = la*P[0]
print(f"A stock-out situation will occur {stock_out:.2f} [unit???]")

A stock-out situation will occur 0.86 [unit???]


#### d) 

In [8]:
# Usikker på disse, må høre med studass
holding_cost = sum(P*[i for i in range(41)]*cH)
print(f"Average holding cost = {holding_cost:.2f}")
shipping_cost = cF*(la/Q)
print(f"Average shipping cost = {shipping_cost:.2f}")
lost_production_cost = cS*P[0]
print(f"Lost production cost = {lost_production_cost:.2f}")

Average holding cost = 32.54
Average shipping cost = 25.00
Lost production cost = 39.33


#### e) 

Since we order a quantity of Q = 20 as soon as we reach r = 20, that is, in state $P_{20}$, we need to caluclate the probability that four days pass ($4\cdot\lambda$ = 20), given that the arrival rate i $\mu$. Let $f_X$ be the probability desnity function of the arrival after an order, which is exponentially distributed with mean value $\frac{1}{\mu}$, that is, the rate is $\mu$ (often denoted $\lambda$ in litterature). Then we simply need to multiply the PDF with the expected delay for all values over 4 (in practice we just go up to 100 days delivery time, that is, a max delay of 96 days). We also allow fractions, i.e., that deliveries may come mid-day.

In [9]:
dist = scipy.stats.expon(mu)
FX = lambda d: dist.pdf(d)*(d-4) # d is the number of days
FX(5)
ave_wait_time, err = scipy.integrate.quad(FX, 4, 100)
print(f"Average wait time in stock out situation = {ave_wait_time:.4f}")

Average wait time in stock out situation = 0.0256


# Task 3 

In [10]:
alpha = 3
muG = 1

#### a) 

For this task, we can apply tricks shows in **7.3.4** in the compendium. The sum of k independant and identically distrbuted exponential random variables (let k=3 and we have the excact situation stated in the problem description) is Erlang-k distibuted. We then think of the delivery process as a 3-stage process where the supplier completes the delivery task in 3 subtasks where each subtask has a gamma distributed completion time with shape parameter $\alpha = 3$ and intensity parameter $\mu_G = 1$. We can model this by adding 2 intermediate states in the transition diagram.

**Markov state transition diagram**

![title](Markov_Task3a.jpg)

As described on page 103, we need to convert the $j_l$ (i.e, $0_1, 0_2, 0_3, 1_1, ...$) notation to a normal index regime, such that we can continue with matrix operations to solve the problem. We apply eq. **7.26:** $$ INDEX(j,l) = kj + l$$

In [11]:
k = 3
def index(j, l):
    return k*j + l

We can now calculate the coefficients of our $A$-matrix using equation **7.25**. Keep in mind that $\mu$ and $\lambda$ is reversed compared to the equation. The resulting matrix size has grown from 41 times 41 to 41k times 41k, where k = 3.

In [12]:
N = 40
num_rows = k*(N+1)
num_cols = num_rows
A = np.zeros((num_rows, num_cols))

# We now repeat 2.a) with using the indexing method 

In [13]:
# Set arrival rates
for j in range(r + 1): # j represents main states
    for l in range(k): # l represents substage
        ind_current = index(j,l)
        if l == 2:
            ind_to = index(j + Q, 0) # Jump Q (20) stages since we have an arrival
        else:
            ind_to = index(j, l + 1) # Go to the next subtask
        A[ind_current, ind_to] = muG
#         print(ind_current, ind_to)
# Set demand rate
for j in range(1, 41): # j represents main states
    for l in range(k): # l represents substage
        ind_current = index(j, l)
        ind_to = index(j-1, l)
        A[ind_current, ind_to] = la 
#         print(ind_current, ind_to)

before = []
for i in range(num_rows):
    before.append(np.sum(A[i]))
    
# Set diagonals such that each row equals 0
for i in range(num_rows):
    for j in range(num_cols):
        if i == j:
            diag_val = -np.sum(A[i])
            A[i,j] = diag_val
# print(A)
after = []
for i in range(num_rows):
    after.append(np.sum(A[i]))


In [14]:
# Control
for i in range(num_rows):
    print(f"Before{i} = {before[i]}")
    print(f"After{i} = {after[i]}")  
    print()

Before0 = 1.0
After0 = 0.0

Before1 = 1.0
After1 = 0.0

Before2 = 1.0
After2 = 0.0

Before3 = 6.0
After3 = 0.0

Before4 = 6.0
After4 = 0.0

Before5 = 6.0
After5 = 0.0

Before6 = 6.0
After6 = 0.0

Before7 = 6.0
After7 = 0.0

Before8 = 6.0
After8 = 0.0

Before9 = 6.0
After9 = 0.0

Before10 = 6.0
After10 = 0.0

Before11 = 6.0
After11 = 0.0

Before12 = 6.0
After12 = 0.0

Before13 = 6.0
After13 = 0.0

Before14 = 6.0
After14 = 0.0

Before15 = 6.0
After15 = 0.0

Before16 = 6.0
After16 = 0.0

Before17 = 6.0
After17 = 0.0

Before18 = 6.0
After18 = 0.0

Before19 = 6.0
After19 = 0.0

Before20 = 6.0
After20 = 0.0

Before21 = 6.0
After21 = 0.0

Before22 = 6.0
After22 = 0.0

Before23 = 6.0
After23 = 0.0

Before24 = 6.0
After24 = 0.0

Before25 = 6.0
After25 = 0.0

Before26 = 6.0
After26 = 0.0

Before27 = 6.0
After27 = 0.0

Before28 = 6.0
After28 = 0.0

Before29 = 6.0
After29 = 0.0

Before30 = 6.0
After30 = 0.0

Before31 = 6.0
After31 = 0.0

Before32 = 6.0
After32 = 0.0

Before33 = 6.0
After33 = 0.0



In [15]:
print(A)

[[-1.  1.  0. ...  0.  0.  0.]
 [ 0. -1.  1. ...  0.  0.  0.]
 [ 0.  0. -1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -5.  0.  0.]
 [ 0.  0.  0. ...  0. -5.  0.]
 [ 0.  0.  0. ...  0.  0. -5.]]


Now that we have the $A$-matrix, we apply the same procedure as in task 2.

In [16]:
# Replace first column with 1's in the A-matrix to get A1
A[:,0] = 1
b = np.zeros(num_rows)
b[0] = 1
# b = np.array([b])

# print(At)
PT = scipy.linalg.solve(np.transpose(A), b.T)
P = PT.T
# print(P)
print(sum(P))

0.9999999999999999


To find the probability of state 0, we summarize the relevant steady states using the index formula with j = 0 and l = 0,1 and 2

In [17]:
j = 0
indexes = []
for l in range(k):
    indexes.append(index(j, l))
print(f"The indexes are {indexes}")
P_0s = [P[ind] for ind in indexes]
P_0_tot = sum(P_0s)
print(P_0_tot)

The indexes are [0, 1, 2]
0.09179723606063121


In [18]:
print(f"Average time we are not able to produce product 2 = {P_0_tot*100:.2f} % of the time")

Average time we are not able to produce product 2 = 9.18 % of the time


#### b) 

In [19]:
stock_out = la*P_0_tot
print(f"A stock-out situation will occur {stock_out:.2f} [unit???]")
# Usikker på disse, må høre med studass
holding_cost = sum([(P[i]+P[i+1]+P[i+2])*i for i in range(41)]*cH)
print(f"Average holding cost = {holding_cost:.2f}")
shipping_cost = cF*(la/Q)
print(f"Average shipping cost = {shipping_cost:.2f}")
lost_production_cost = cS*P_0_tot
print(f"Lost production cost = {lost_production_cost:.2f}")

A stock-out situation will occur 0.46 [unit???]
Average holding cost = 44.61
Average shipping cost = 25.00
Lost production cost = 21.11


#### c) 

For this task, we apply the same methodology as in **Problem 8.2** in exercise 6.

In [25]:

L = alpha/mu # Lead time, by the gamma distribution (eq. 2.34)
L_Var = alpha/mu**2 # Variance lead time, by the gamma distribution (eq. 2.35)
L_SD = math.sqrt(L_Var)
Delta = la # demand rate, we assume its normally distributed with sigma = 1
D_Var = 1
D_SD = 1
cS = 230
cH = 2
cF = 100
cP = 10 # Assume that the lost sale penalty is just a loss of profit
print(L)
print(L_SD)
print(Delta)


9.0
5.196152422706632
5


First, we write down equations (8.17) and (8.19) as functions of r and Q respectively. In order to do that we need an expression for B(r), and we use equation (8.13) since the demand is normally distributed. Recall that $\Phi(u) = F_U(u)$ and $\phi(u) = f_U(u)$, that is Phi and phi is the standard normally distributed cumulative density function and probability density function respectively

In [21]:
Phi = lambda u: scipy.stats.norm().cdf(u)
phi = lambda u: scipy.stats.norm().pdf(u)

In [22]:
# Equation 8.13
calc_B = lambda r: (Delta -r)*(1 - Phi((r - Delta)/D_SD)) + D_SD*phi((r - Delta)/D_SD)

In [23]:
# Equation 8.17
calc_Q = lambda r: math.sqrt(2*Delta*(cF + cP*calc_B(r))/cH)
# Equation 8.19
ILS = 1 # Assuming lost_sales
def calc_r(Q):
    norm =scipy.stats.norm(Delta, D_SD)
    frac = (Delta*cP + (ILS - 1)*cH*Q)/(Delta*cP + ILS*cH*Q)
    return norm.ppf(frac)

In [24]:
# Iterative scheme
tol = 1e-6 #Tolerance
i = 1
Q = math.inf
r = 0 #These two are really random. I use0 and math.inf to get 
# an unachievable difference for the first iteration
diffR = []
diffQ = []
while True:
    print(f"Iteration: {i}")
    prev_r = r
    prev_Q = Q
    if i == 1:
        Q = math.sqrt(2*Delta*cF/cH) # First step, B(r) = 0
    else:
        Q = calc_Q(r)
    r = calc_r(Q)
    diffR.append(abs(r - prev_r))
    diffQ.append(abs(Q - prev_Q))
    print(f"r = {r:.2f}, Q = {Q:.2f}")
    if diffR[-1] < tol and diffQ[-1] < tol:
        print(f"----Solution found at r = {r:.4f}, Q = {Q:.4f}")
        break
    
    i += 1
    if i >= 10000:
        break
        
#     print(f"r = {r:.2f}, Q = {Q.:2f} ok")

Iteration: 1
r = 5.07, Q = 22.36
Iteration: 2
r = 5.06, Q = 22.77
Iteration: 3
r = 5.06, Q = 22.77
Iteration: 4
r = 5.06, Q = 22.77
Iteration: 5
r = 5.06, Q = 22.77
Iteration: 6
r = 5.06, Q = 22.77
----Solution found at r = 5.0585, Q = 22.7710


# Task 4 

Skip denne? Ser ikke veldig relevant ut for eksamen.